In [4]:
### 모듈로 얻은 데이터 feature 뽑고 10개로 자르는 코드

import os
import pandas as pd
import numpy as np
import scipy
import torch
import neurokit2 as nk
import scipy.io as sio
import matplotlib.pyplot as plt
import csv

load_home = f'/tf/ssssun/ECG/datas/raw/ECG_1101_JJH.csv'
save_home = f'/tf/ssssun/ECG/datas/input/ECG_10sec_1101_feature1.csv'
rate = 200


data = pd.read_csv(load_home)
data = np.array(data)

suspicious_index = np.where(data >= 3)[0]

for si in suspicious_index:
    # 마지막 요소
    if (si == len(data)):
        data[si] = data[si - 1]
    
    # 의심 값 중에 앞뒤 차이가 큰 값이면 노이즈로 판단
    if (data[si] - data[si - 1] > 2.5) and (data[si] - data[si + 1] > 2.5):        
        count = 1
        # 다음 값도 3이상 (연속된 노이즈)
        while (data[si + count] >= 3):
            count += 1

        # 첫 요소
        if (si == 0):
            data[si] = data[si + count]

        else:
            median = (data[si-1] + data[si + count]) / 2
            data[si] = median
        
        
# for ni in suspicious_index:
#     print(ni, data[ni])

data = data.reshape(-1)
data_home = nk.ecg_clean(data, sampling_rate=rate) #filtering
# data_home = data_home[1000:len(data_home)-10000-np.remainder(len(data_home),10005)]  #데이터의 앞,뒤 자름

_, rpeaks = nk.ecg_peaks(data_home, sampling_rate=rate) #rpeak 추출
_, waves_peak = nk.ecg_delineate(data_home, rpeaks, sampling_rate=rate, method="peak") #p,q,s,t 추출


R_peaks=[]
T_peaks=[]
P_peaks=[]
Q_peaks=[]
S_peaks=[]
R_peaks = rpeaks['ECG_R_Peaks']     #dict에서 결과 numpy만 가져옴
T_peaks = waves_peak['ECG_T_Peaks']
P_peaks = waves_peak['ECG_P_Peaks']
Q_peaks = waves_peak['ECG_Q_Peaks']
S_peaks = waves_peak['ECG_S_Peaks']

amp_feature = []
for i in range(len(R_peaks)-1):
    feature=[]
    if((not Q_peaks) | (not S_peaks) | (not P_peaks) | (not T_peaks)):  #데이터 빈었는지
        continue
    else: #데이터의 nan 있는지 확인
        if((not(np.isnan(R_peaks[i]))) & (not(np.isnan(Q_peaks[i]))) & (not(np.isnan(S_peaks[i]))) & (not (np.isnan(P_peaks[i]))) & (not (np.isnan(T_peaks[i])))):
            PR_amp = data_home[R_peaks[i]]-data_home[P_peaks[i]]
            QR_amp = data_home[R_peaks[i]]-data_home[Q_peaks[i]]
            RS_amp = data_home[R_peaks[i]]-data_home[S_peaks[i]]
            RT_amp = abs(data_home[R_peaks[i]]-data_home[T_peaks[i]])
            PR_inter = abs(P_peaks[i]-R_peaks[i])
            QR_inter = (abs(R_peaks[i]-Q_peaks[i]))
            RS_inter = (abs(R_peaks[i]-S_peaks[i]))
            RT_inter = abs(T_peaks[i]-R_peaks[i])
            RR_inter = R_peaks[i+1] - R_peaks[i]
            ST_inter = abs(T_peaks[i]-S_peaks[i])

            feature.append(PR_amp)
            feature.append(QR_amp)
            feature.append(RS_amp)
            feature.append(RT_amp)
            feature.append(PR_inter)
            feature.append(QR_inter)
            feature.append(RS_inter)
            feature.append(RT_inter)
            feature.append(RR_inter)
            feature.append(ST_inter)
            amp_feature.append(feature)   #(추출한 peak수, 10)으로 저장


print(len(amp_feature))

amp_feature = amp_feature[0:10]  #길이 10으로 자름

with open(save_home, 'w') as file:
    write = csv.writer(file)
    write.writerows(amp_feature)
    

12
